<a href="https://colab.research.google.com/github/Seb012007/TradingBot/blob/main/TestBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deps : 

In [ ]:
!pip install python-binance ta nest-asyncio

Code


In [73]:
import pandas as pd
from binance.client import Client
import ta
from binance.enums import *
import asyncio
from binance import AsyncClient, BinanceSocketManager
import nest_asyncio
import threading
import time

api_key = "6OUMRj4NHlaZHXP35hq6iSgJBv7Q32PSlEeYpdHTUJEQp5mmJUJF4F0Urj7HdWYT"
api_secret = "J9WNE2eLWSWYqt1Fp3AaSiJfE4Q9ds4BzrfQmsK1sCHvorAJz90a2w2uZ9jqUrOM"
asset1 = 'BNB'
asset2 = 'USDT'

client = Client(api_key, api_secret,testnet=True)

klinesT = Client(api_key, api_secret).get_historical_klines(asset1+asset2, Client.KLINE_INTERVAL_1HOUR, "01 January 2021")
df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])

del df['ignore']
del df['close_time']
del df['quote_av']
del df['trades']
del df['tb_base_av']
del df['tb_quote_av']

df['close'] = pd.to_numeric(df['close'])
df['high'] = pd.to_numeric(df['high'])
df['low'] = pd.to_numeric(df['low'])
df['open'] = pd.to_numeric(df['open'])


df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')

del df['timestamp']
df['EMA9'] = ta.trend.ema_indicator(df['close'], 9)
df['EMA18'] = ta.trend.ema_indicator(df['close'], 18)


lastIndex = df.first_valid_index()
print(asset1 +" At Start : " + client.get_asset_balance(asset=asset2)["free"])
print(asset2 +" At Start : " + client.get_asset_balance(asset=asset1)["free"])
n_asset_2 = pd.to_numeric(client.get_asset_balance(asset=asset2)["free"])
n_asset_1 = pd.to_numeric(client.get_asset_balance(asset=asset1)["free"])

buy = True

for index, row in df.iterrows():


  if df['EMA9'][lastIndex] > df['EMA18'][lastIndex] and n_asset_2 > 10 and buy == True:
    time.sleep(1)
    n_asset_2 = pd.to_numeric(client.get_asset_balance(asset=asset2)["free"])
    n_asset_1 = pd.to_numeric(client.get_asset_balance(asset=asset1)["free"])
    print("Buy "+ asset1 +" at ",df['close'][index],'$ the', index)
    quantity=round((n_asset_2/10000)*5,1)
    print(asset1+" quantity : "+str(quantity))
    if(quantity) > 0.1:
      order = client.order_market_buy(
        symbol=asset1+asset2,
        quantity=quantity)
    buy = False
    crossUp = False

  if df['EMA9'][lastIndex] < df['EMA18'][lastIndex] and n_asset_1 > 0.0001 and buy == False:
    time.sleep(1)
    n_asset_2 = pd.to_numeric(client.get_asset_balance(asset=asset2)["free"])
    n_asset_1 = pd.to_numeric(client.get_asset_balance(asset=asset1)["free"])
    print("Sell "+ asset1 +" at ",df['close'][index],'$ the', index)
    quantity = round((n_asset_1/10000)*5,1)
    print(asset1+" quantity : "+str(quantity))
    symbol = asset1+asset2
    if(quantity) > 0.1:
      order = client.order_market_sell(
          symbol=symbol,
          quantity=quantity)
    buy = True

  lastIndex = index

print(asset1 +" At End : " + client.get_asset_balance(asset=asset2)["free"])
print(asset2 +" At End : " + client.get_asset_balance(asset=asset1)["free"])



{'e': 'trade', 'E': 1643665670334, 's': 'BNBBTC', 't': 177905930, 'p': '0.00991000', 'q': '0.55700000', 'b': 1354693486, 'a': 1354693413, 'T': 1643665670333, 'm': False, 'M': True}
{'e': 'trade', 'E': 1643665670334, 's': 'BNBBTC', 't': 177905931, 'p': '0.00991000', 'q': '6.52900000', 'b': 1354693486, 'a': 1354693417, 'T': 1643665670333, 'm': False, 'M': True}
{'e': 'trade', 'E': 1643665670408, 's': 'BNBBTC', 't': 177905932, 'p': '0.00991000', 'q': '1.68400000', 'b': 1354693490, 'a': 1354693417, 'T': 1643665670408, 'm': False, 'M': True}
{'e': 'trade', 'E': 1643665670408, 's': 'BNBBTC', 't': 177905933, 'p': '0.00991000', 'q': '0.28600000', 'b': 1354693490, 'a': 1354693453, 'T': 1643665670408, 'm': False, 'M': True}
BNB At Start : 727.42111946
USDT At Start : 966.40000000
Buy BNB at  37.5989 $ the 2021-01-01 18:00:00
BNB quantity : 0.4
Sell BNB at  37.401 $ the 2021-01-01 20:00:00
BNB quantity : 0.5
Buy BNB at  37.9948 $ the 2021-01-02 14:00:00
BNB quantity : 0.4
Sell BNB at  40.0211 $ t

Test async

In [68]:
import pandas as pd
from binance.client import Client
import ta
from binance.enums import *
import asyncio
from binance import AsyncClient, BinanceSocketManager
import nest_asyncio
import threading
import time

nest_asyncio.apply()

api_key = "6OUMRj4NHlaZHXP35hq6iSgJBv7Q32PSlEeYpdHTUJEQp5mmJUJF4F0Urj7HdWYT"
api_secret = "J9WNE2eLWSWYqt1Fp3AaSiJfE4Q9ds4BzrfQmsK1sCHvorAJz90a2w2uZ9jqUrOM"
asset1 = 'BNB'
asset2 = 'USDT'

async def main():
    client = await AsyncClient.create(api_key, api_secret,testnet=True)
    bm = BinanceSocketManager(client)
    # start any sockets here, i.e a trade socket
    ts = bm.user_socket()
    # then start receiving messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            print(res)

    await client.close_connection()

def test():

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

x = threading.Thread(target=test)